In [ ]:
import itertools, collections

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from numpy.linalg import *

import collections
import re
import time
import itertools
from collections import defaultdict, Counter, OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
import string

from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
reviews_df = pd.read_csv("IMDB Dataset.csv")
reviews_df.sentiment = reviews_df.sentiment.apply(lambda x: 1 if x == "positive" else 0)

FileNotFoundError: ignored

In [ ]:
reviews_df.head()

,Unnamed: 0,review,sentiment
0,0,one of the other reviewers has mentioned that ...,1
1,1,a wonderful little production the filming tech...,1
2,2,i thought this was a wonderful way to spend ti...,1
3,3,basically theres a family where a little boy j...,0
4,4,petter matteis love in the time of money is a ...,1


## Preprocessing

In [ ]:
reviews_df['review'] = reviews_df['review'].str.replace('<br /><br />', '')
reviews_df['review'] = reviews_df['review'].str.replace('\t', '')

In [ ]:
# converting odd characters to corresponding english characters
character_mappings = {'é':'e',
                   'è':'e',
                   'à':'a',
                   'ô':'o',
                   'û':'u',
                   'ó':'o',
                   'ü':'u',
                   'â':'a',
                   'í':'i',
                   'ñ':'n',
                   'ë':'e',
                   'ä':'a',
                   'á':'a',
                   'ã':'a',
                   'å':'a',
                   'ï':'i',
                   'ú':'u',
                   'ö':'o',
                   'ì':'i',
                   'Æ':'Ae',
                   'Á':'A',
                   'ê':'e',
                   'î':'i',
                   'ç':'c',
                   'Õ':'O',
                   'Ö':'O',
                   'Ï':'I',
                   'ù':'u',
                   'É':'E',
                   'Ð':'D',
                   'õ':'o',
                   'ō':'o',
                   'ý':'y',
                   'ı':'i',
                   'ğ':'g',
                   'ð':'o',
                   'Ó':'O',
                   'Ü':'U',
                   'Ä':'A',
                   'ß':'B',
                   'Å':'A',
                   'Ã':'A',
                   'Â':'A',
                   'À':'A',
                   'Ż':'Z',
                   'ò':'o',
                   'Ê':'E',
                   'æ':'ae',
                   'Þ':'p',
                   'Ç':'C',
                   'È':'E',
                   '©':'c',
                   'ª':'a',
                   'Ø':'O'}

for old_char, new_char in character_mappings.items():
    reviews_df['review'] = reviews_df['review'].str.replace(old_char, new_char)

In [ ]:
# removing undefined characters
unknown_characters = {'\x97', '\x96', '\x85','\x84','§', '\x91','\xa0','▼','\x95','\x80','\x10','\x9a','\x08','º','°','\x9e','\x8e','\x8d','\xad','\uf0b7','¤'}

for character in unknown_characters:
    reviews_df['review'] = reviews_df['review'].str.replace(character, '')

In [ ]:
reviews_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
from dateutil import parser
import html
# declaration of helper sets
s = stopwords.words('english')
p = string.punctuation

#Helper function to identify dates
def is_not_date(string):
  try:
    parser.parse(string)
    return False
  except:
    return True

# function to clean a comment
def clean_review(review):
  # convert html chars into text and remove HTML tags
  review = html.unescape(review)

  # convert to lower case
  # review = ' '.join(x.lower() for x in review.split())

  #remove stopwords
  # review = ' '.join(x for x in review.split() if x not in s and x.isalpha())

  # remove punctuation
  review = ''.join(x for x in review if x not in p)

  # remove URL
  review = re.sub(r"https?://\S+|www\.\S+", '', review).strip()

  # remove hashtags
  review = re.sub(r'\B(\#[a-zA-Z]+\b)', '', review).strip()

  #Remove date and time
  review = ' '.join(x for x in review.split() if is_not_date(x))

  return review

In [ ]:
#this takes a while to run
reviews_df['review'] = reviews_df.review.apply(lambda x: clean_review(x))

In [ ]:
reviews_df.to_csv("reviews_df_cleaned.csv")

In [ ]:
reviews_df = pd.read_csv("reviews_df_cleaned.csv")

In [ ]:
reviews_df.review = reviews_df.review.str.lower()

In [ ]:
reviews_df.head()

,Unnamed: 0,review,sentiment
0,0,one of the other reviewers has mentioned that ...,1
1,1,a wonderful little production the filming tech...,1
2,2,i thought this was a wonderful way to spend ti...,1
3,3,basically theres a family where a little boy j...,0
4,4,petter matteis love in the time of money is a ...,1


In [ ]:
review_str = reviews_df.review.apply(lambda s: s.split()).values
vocab={}

for review in tqdm(review_str):
    for r in review:
        if r not in vocab.keys():
          vocab[r] = 1
        else:
          vocab[r] += 1


100%|██████████| 50000/50000 [00:05<00:00, 8673.39it/s] 


In [ ]:
id_to_word = dict(enumerate(vocab))
word_to_id = {v:k for k, v in id_to_word.items()}

In [ ]:
def convert_to_padded(sent, max_len, trim_len):
  """
  This basically converts given sentence into a format that can be used by the LStM
  """

  padded_arr = np.zeros(max_len).astype(int)

  padded_arr[list(range(0, len(sent.split(' '))))] = [word_to_id[w] for w in sent.split(' ')]

  return np.append(padded_arr[:trim_len], min(trim_len, len(sent.split(' '))))

In [ ]:
#Lets transform data to padded versions

max_len = max(list(reviews_df.review.apply(lambda x: len(x.split(' ')))))
trimmed_len = max_len
X_padded = np.zeros((len(reviews_df), trimmed_len+1))
y = np.asarray(reviews_df.sentiment.tolist())
seq_l = np.zeros(len(reviews_df)).astype(int)

for i in tqdm(range(len(reviews_df))):
  padded_arr = convert_to_padded(reviews_df.iloc[i].review, max_len, trimmed_len)
  X_padded[i] = padded_arr



100%|██████████| 50000/50000 [00:16<00:00, 3090.73it/s]


In [ ]:
#Lets split the data into training and testing
X_train, X_rest, y_train, y_rest = train_test_split(X_padded, y, test_size = 0.3)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size = 0.5)


In [ ]:
X_train, X_val, X_test = X_train.astype(int), X_val.astype(int), X_test.astype(int)

In [ ]:
seq_l_train = X_train[:, -1]
seq_l_test = X_test[:, -1]
seq_l_val = X_val[:, -1]

X_train = X_train[:, :-1]
X_test = X_test[:, :-1]
X_val = X_val[:, :-1]

In [ ]:
len(X_test[0])

2441

In [ ]:
#We'll make a quick pytorch dataset to use later
class reviewPytorchDataset(Dataset):
  def __init__(self, X, y, seq_l):
    self.X = X
    self.y = y
    self.seq_l = seq_l

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    return self.X[idx], self.y[idx], self.seq_l[idx]

In [ ]:
train_dataset = reviewPytorchDataset(X_train, y_train, seq_l_train)
test_dataset = reviewPytorchDataset(X_test, y_test, seq_l_test)
val_dataset = reviewPytorchDataset(X_val, y_val, seq_l_val)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

## Getting embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget -nc https://huggingface.co/stanfordnlp/glove/resolve/main/glove.840B.300d.zip
!unzip glove.840B.300d.zip
!ls -lat
glove_file = "glove.840B.300d.txt"

--2023-12-08 07:05:32--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.840B.300d.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/f47355dd5b267bd10f08671e513790690233c76a9ffd73aa915d78f894a8912e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27glove.840B.300d.zip%3B+filename%3D%22glove.840B.300d.zip%22%3B&response-content-type=application%2Fzip&Expires=1702278332&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjI3ODMzMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zdGFuZm9yZG5scC9nbG92ZS9mNDczNTVkZDViMjY3YmQxMGYwODY3MWU1MTM3OTA2OTAyMzNjNzZhOWZmZDczYWE5MTVkNzhmODk0YTg5MTJlP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=cKZLnD

In [ ]:
!cp glove.840B.300d.txt /content/drive/MyDrive/519_temp


In [ ]:
def get_glove_mapping(vocab, file):


    glove_map = {}
    with open(file,'rb') as fi:
        for l in tqdm(fi):
            try:
                #### STUDENT CODE HERE ####
                ## only include vectors that are found in the vocabulary.
                line_split = str(l.decode('utf8')).split(' ')
                if line_split[0] in vocab:
                  glove_map[line_split[0]] = [np.float(v) for v in line_split[1:]]

                #### STUDENT CODE ENDS HERE ####
            except:
                #some lines have urls, we don't need them.
                pass
    return glove_map

In [ ]:
glove_map = get_glove_mapping(vocab, '/content/drive/MyDrive/519_temp/glove.840B.300d.txt')

0it [00:00, ?it/s]<ipython-input-18-b275e0217be7>:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  glove_map[line_split[0]] = [np.float(v) for v in line_split[1:]]
2196018it [01:43, 21115.61it/s]


In [ ]:
import pickle

with open('/content/drive/MyDrive/519_temp/glove_map.pkl', 'wb') as f:
    pickle.dump(glove_map, f)

In [ ]:
import pickle

with open('/content/drive/MyDrive/519_temp/glove_map.pkl', 'rb') as f:
    glove_map = pickle.load(f)

In [ ]:
glove_map['drawn']

[0.14287,
 -0.31072,
 0.39894,
 -0.13926,
 -0.056802,
 -0.093931,
 -0.073266,
 0.73137,
 -0.2503,
 2.3727,
 0.13788,
 0.39433,
 0.070549,
 -0.061729,
 -0.091478,
 -0.0020465,
 -0.43601,
 0.70205,
 0.084257,
 -0.29981,
 0.063431,
 -0.58384,
 -0.41866,
 0.24423,
 0.48236,
 -0.064663,
 0.23805,
 -0.01991,
 -0.325,
 -0.26264,
 -0.45733,
 -0.085568,
 -1.0164,
 0.044419,
 -0.15447,
 -0.16698,
 -0.061927,
 0.091379,
 -0.15283,
 -0.50401,
 -0.32298,
 0.049078,
 -0.16023,
 -0.24246,
 -0.028448,
 -0.35837,
 -0.094088,
 -0.061352,
 0.35402,
 -0.15013,
 0.091884,
 0.013132,
 -0.63221,
 -0.27103,
 -0.072217,
 0.085832,
 0.24065,
 -0.016327,
 0.22608,
 0.00053107,
 0.038915,
 0.30714,
 0.16135,
 -0.13731,
 -0.31002,
 0.51496,
 0.11979,
 0.034811,
 0.47813,
 0.12617,
 -0.14136,
 -0.12586,
 0.45562,
 0.34497,
 0.10185,
 0.0014311,
 -0.23317,
 -0.5072,
 -0.1536,
 0.21684,
 -0.15858,
 -0.011117,
 -0.087891,
 -0.050092,
 0.26141,
 -0.38928,
 0.17328,
 0.39166,
 0.4299,
 0.080008,
 0.26108,
 -0.12347,
 -0

In [ ]:
n_embed = len(vocab.items())
d_embed = 300

In [ ]:
#Creating map based on word_to_id
def get_embedding_matrix(n_embed, d_embed, glove_map, randomize_init = False):
    """
    Initialize the embedding matrix

    INPUT:
    n_embed         - size of the dictionary of embeddings
    d_embed         - the size of each embedding vector
    glove_map       - the map you created storing all of embeddings you will need from GloVE
    randomize_init  - if True, ignore the embeddings from glove_map and intilize all embeddings to random guassian noise (np.random.normal will be useful).

    OUTPUT:
    embedding_matrix  - a numpy matrix of mapping from word id to embedding

    """
    #### STUDENT CODE HERE ####
    if randomize_init == True:
      return np.random.normal(size = (n_embed, d_embed))
    else:
      embedding_matrix = np.zeros((n_embed, d_embed))
      for w, id in tqdm(word_to_id.items()):
        if w in glove_map.keys():
          embedding_matrix[id] = glove_map[w]
        else:
          embedding_matrix[id] = np.random.normal(size = d_embed)
      return embedding_matrix

    #### STUDENT CODE ENDS HERE ####


In [ ]:
#Creating embedding layer
embedding_matrix = get_embedding_matrix(n_embed, d_embed, glove_map, randomize_init = False)


100%|██████████| 220637/220637 [00:03<00:00, 69031.02it/s]


In [ ]:
embedding_matrix.shape

(220637, 300)

In [ ]:
def create_emb_layer(embedding_matrix, non_trainable=False):
    """
    Create the embedding layer

    INPUT:
    embedding_matrix  - matrix of mapping from word id to embedding
    non_trainable   - Flag for whether the embedding matrix should be trained.
                      If it is set to True, don't update the gradients

    OUTPUT:
    emb_layer       - embedding layer

    """
    #### STUDENT CODE HERE ####
    emb_pretrained = torch.FloatTensor(embedding_matrix)
    emb_layer = nn.Embedding.from_pretrained(embeddings = emb_pretrained, \
                                             freeze = non_trainable)
    #### STUDENT CODE ENDS HERE ####

    return emb_layer

## Defining the LSTM network

In [ ]:
import random as random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import LSTM, GRU

class LSTM_Classifier(nn.Module):

    def __init__(self,
                 n_embed=20000,
                 d_embed=300,
                 d_hidden=150,
                 freeze_emb = False,
                 bidirectional = False,
                 aggregation = "last"
                 ):
        super(LSTM_Classifier, self).__init__()

        self.d_hidden = d_hidden
        self.bidrectional = bidirectional

        self.aggregation = aggregation

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.embed = create_emb_layer(embedding_matrix,freeze_emb)

        self.recurrent_cell = nn.LSTM(input_size = d_embed, hidden_size = d_hidden, \
                                 batch_first = True, bidirectional = bidirectional)

        self.output_cell = nn.Linear(d_hidden, 1)


    def forward(self, text, seq_lengths):
        seq_lengths = seq_lengths.squeeze()

        if self.aggregation == "last":

          #avoiding mask for now
          x = text
          x = self.embed(x)
          x = pack_padded_sequence(x, seq_lengths, batch_first = True, enforce_sorted = False)

          pred = None

          if self.bidrectional == False:

            out, (hidden, cell) = self.recurrent_cell(x)
            pred = self.output_cell(hidden[0])

          else:

            out, (hidden, cell) = self.recurrent_cell(x)
            hidden_avg = hidden.sum(axis = 0)
            pred = self.output_cell(hidden_avg)

          return pred

        elif self.aggregation == "mean":
          x = text

          x = self.embed(x)
          x = pack_padded_sequence(x, seq_lengths, batch_first = True, enforce_sorted = False)

          pred = None
          if self.bidrectional == False:

            out, (hidden, cell) = self.recurrent_cell(x)

            out_padded, _ = pad_packed_sequence(out, batch_first = True)

            out_means = torch.mean(out_padded, dim = 1)

            pred = self.output_cell(out_means)

          else:

            out, (hidden, cell) = self.recurrent_cell(x)
            out_padded, _ = pad_packed_sequence(out, batch_first = True)

            out_padded_avg = (out_padded[:, :, : int(out_padded.shape[2] / 2)] + \
                              out_padded[:, :, int(out_padded.shape[2] / 2):]) / 2

            out_means = torch.mean(out_padded_avg, dim = 1)
            pred = self.output_cell(out_means)


          return pred

        else:

          raise Exception("Unsupported aggregation format")



## Training the network

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
def evaluate_model(y_pred, y_test):

  return accuracy_score(y_pred, y_test), recall_score(y_pred, y_test), \
        precision_score(y_pred, y_test), f1_score(y_pred, y_test)

In [ ]:
def train(model, train_iter, epochs = 5, lr = .005, val = "val", weight_decay = 1e-5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.BCEWithLogitsLoss()
    model = model
    model.to(device)
    print(
        'epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |')

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)

    save_path = "best_model"
    best_acc = 0

    iterations = 0
    last_val_iter = 0
    train_loss = 0
    start = time.time()
    _save_ckp = ''
    accuracy_history = []
    for epoch in range(epochs):
        n_correct, n_total, train_loss = 0, 0, 0
        last_val_iter = 0

        for batch_idx, batch in enumerate(train_iter):
            # switch model to training mode, clear gradient accumulators
            model.train()
            optimizer.zero_grad()
            iterations += 1

            X, y, seq_l = batch
            X = X.to(device)
            y = y.unsqueeze(1).to(device).float()
            seq_l = seq_l.unsqueeze(1).long()

            answer = model(X, seq_l)

            loss = criterion(answer, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            answer[answer >= 0.5] = 1
            answer[answer < 0.5] = 0

            if iterations > 0 and iterations % 100 == 0:
              acc = accuracy_score(answer.cpu().detach().numpy(), y.cpu().detach().numpy())
              print("training_acc = " + str(acc))
              print("epoch = " + str(epoch))
              print("loss = " + str(loss.item()))

              # if current_accuracy > best_acc:
              #   best_acc = current_accuracy
              #   torch.save(model.state_dict(), save_path)
              #   _save_ckp = '*'
                #save model so you can load it later

            # print('\r {:4d} | {:4d}/{} | {:.4f} | {:.4f} |'.format(
            #     epoch, batch_size * (batch_idx + 1), len(train_dataset), loss.item(),
            #            train_loss / (iterations - last_val_iter)), end='')


    # model.load_state_dict(torch.load(save_path))
    with torch.no_grad():
      model.eval()


      if val == "val":

        y_pred = model(torch.from_numpy(X_val).to('cuda'), seq_l_val)
        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_val).view(-1, 1))
      elif val == "test":
        y_pred = model(torch.from_numpy(X_test).to('cuda'), seq_l_test)
        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_test).view(-1, 1))
      else:
        pass

    return acc, recall, precision, f1

In [ ]:
len(X_test)

7500

In [ ]:
seq_l_test

array([112, 409, 312, ..., 223,  46, 221])

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
embedding_matrix = get_embedding_matrix(n_embed, d_embed, glove_map, randomize_init = False)
model = LSTM_Classifier(n_embed=n_embed, d_embed=d_embed, d_hidden=100, bidirectional = True, aggregation = "mean")
acc, recall, precision, f1 = train(model, train_loader, epochs = 3, lr = .0005, val = "test")

100%|██████████| 220637/220637 [00:03<00:00, 65530.26it/s]


epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.65625
epoch = 0
loss = 0.6126601696014404
training_acc = 0.65625
epoch = 0
loss = 0.5076858401298523
training_acc = 0.9375
epoch = 0
loss = 0.43021970987319946
training_acc = 0.875
epoch = 0
loss = 0.39499735832214355
training_acc = 0.65625
epoch = 0
loss = 0.6084170341491699
training_acc = 0.6875
epoch = 0
loss = 0.5991591215133667
training_acc = 0.84375
epoch = 0
loss = 0.43040746450424194
training_acc = 0.8125
epoch = 0
loss = 0.337202787399292
training_acc = 0.8125
epoch = 0
loss = 0.39543765783309937
training_acc = 0.875
epoch = 0
loss = 0.37739497423171997
training_acc = 0.84375
epoch = 1
loss = 0.21924279630184174
training_acc = 0.84375
epoch = 1
loss = 0.35861748456954956
training_acc = 0.9375
epoch = 1
loss = 0.21763838827610016
training_acc = 1.0
epoch = 1
loss = 0.1471846103668213
training_acc = 0.875
epoch = 1
loss = 0.25902414321899414
training_acc = 0.90625
epoch = 1
loss = 0

OutOfMemoryError: ignored

In [ ]:
acc_l, recall_l, precision_l, f1_l = [], [], [], []
for i in range(7):

  with torch.no_grad():
      model.eval()

      y_pred = model(torch.from_numpy(X_test[i*1000:(i+1)*1000]).to('cuda'), seq_l_test[i*1000:(i+1)*1000])
      # print(y_pred)
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0

      acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_test[i*1000:(i+1)*1000]).view(-1, 1))
      acc_l.append(acc)
      recall_l.append(recall)
      precision_l.append(precision)
      f1_l.append(f1)
print(np.mean(acc_l), np.mean(recall_l), np.mean(precision_l), np.mean(f1_l))

0.888857142857143 0.9086106176805383 0.8689257747855837 0.8881376560116204


In [ ]:
acc_l, recall_l, precision_l, f1_l = [], [], [], []
for i in range(35):

  with torch.no_grad():
      model.eval()

      y_pred = model(torch.from_numpy(X_train[i*1000:(i+1)*1000]).to('cuda'), seq_l_train[i*1000:(i+1)*1000])
      # print(y_pred)
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0

      acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_train[i*1000:(i+1)*1000]).view(-1, 1))
      acc_l.append(acc)
      recall_l.append(recall)
      precision_l.append(precision)
      f1_l.append(f1)
print(np.mean(acc_l), np.mean(recall_l), np.mean(precision_l), np.mean(f1_l))

0.9738857142857142 0.9836524410989846 0.9636785185819351 0.9734628933007843


In [ ]:
len(np.where(seq_l_test > 200)[0])

2953

In [ ]:
X_test_long = X_test[list(np.where(seq_l_test > 200)[0])]
seql_test_long = seq_l_test[list(np.where(seq_l_test > 200)[0])]
y_test_long = y_test[list(np.where(seq_l_test > 200)[0])]

In [ ]:
seql_test_long

array([409, 312, 325, ..., 835, 223, 221])

In [ ]:
acc_l, recall_l, precision_l, f1_l = [], [], [], []
for i in range(3):

  with torch.no_grad():
      model.eval()
      print((i+1)*1000)
      y_pred = model(torch.from_numpy(X_test_long[i*1000:(i+1)*1000]).to('cuda'), seql_test_long[i*1000:(i+1)*1000])
      # print(y_pred)
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0

      acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_test_long[i*1000:(i+1)*1000]).view(-1, 1))
      acc_l.append(acc)
      recall_l.append(recall)
      precision_l.append(precision)
      f1_l.append(f1)
print(np.mean(acc_l), np.mean(recall_l), np.mean(precision_l), np.mean(f1_l))

1000
2000
3000
0.8884099335431969 0.8882681291570712 0.8912515819507948 0.889735957172487


In [ ]:
acc_l, recall_l, precision_l, f1_l = [], [], [], []
for i in range(7):

  with torch.no_grad():
      model.eval()

      y_pred = model(torch.from_numpy(X_val[i*1000:(i+1)*1000]).to('cuda'), seq_l_val[i*1000:(i+1)*1000])
      # print(y_pred)
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0

      acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_val[i*1000:(i+1)*1000]).view(-1, 1))
      acc_l.append(acc)
      recall_l.append(recall)
      precision_l.append(precision)
      f1_l.append(f1)
print(np.mean(acc_l), np.mean(recall_l), np.mean(precision_l), np.mean(f1_l))

0.8971428571428571 0.9078112571109598 0.8842557232906932 0.8956296997810554


In [ ]:
len(X_train)

35000

In [ ]:
y_pred.to('cpu').detach().numpy()

array([[0.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
torch.from_numpy(y).view(-1, 1)

tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]])

In [ ]:
np.where(y_test == 0)

(array([    3,     7,     8, ..., 12489, 12493, 12494]),)

In [ ]:
len(X_test)

12500

In [ ]:
len([p for p in model.parameters()])

7

In [ ]:
model

LSTM_Classifier(
  (embed): Embedding(220637, 300)
  (recurrent_cell): LSTM(300, 300, batch_first=True)
  (output_cell): Linear(in_features=300, out_features=1, bias=True)
)

## Experiments

### Hyperparameter tuning

d_hidden
lr

In [ ]:
d_hidden = [50, 100, 150]
lr = [0.0001, 0.0005, 0.001]
hidden_df = []
lr_df = []
acc_df = []
recall_df = []
precision_df = []
f1_df = []

for d in d_hidden:
  for l in tqdm(lr):

    model = LSTM_Classifier(n_embed=n_embed, d_embed=d_embed, d_hidden=d, bidirectional = False, aggregation = "last")
    acc, recall, precision, f1 = train(model, train_loader, epochs = 3, lr = l)
    hidden_df.append(d)
    lr_df.append(l)
    acc_df.append(acc)
    recall_df.append(recall)
    precision_df.append(precision)
    f1_df.append(f1)


tuning_df = pd.DataFrame({"Num hidden": hidden_df, "Learning Rate": lr_df, "Acc": acc_df, "Recall": recall_df, \
                          "Precision": precision_df, "F1": f1_df})

  0%|          | 0/3 [00:00<?, ?it/s]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.5
epoch = 0
loss = 0.6808536052703857
training_acc = 0.3125
epoch = 0
loss = 0.6828982830047607
training_acc = 0.53125
epoch = 0
loss = 0.6867831349372864
training_acc = 0.59375
epoch = 0
loss = 0.6848427057266235
training_acc = 0.40625
epoch = 0
loss = 0.6890532374382019
training_acc = 0.59375
epoch = 0
loss = 0.623481273651123
training_acc = 0.75
epoch = 0
loss = 0.5188906788825989
training_acc = 0.65625
epoch = 0
loss = 0.5536943674087524
training_acc = 0.8125
epoch = 0
loss = 0.4960361421108246
training_acc = 0.65625
epoch = 0
loss = 0.6114431619644165
training_acc = 0.78125
epoch = 1
loss = 0.5292783975601196
training_acc = 0.71875
epoch = 1
loss = 0.48631751537323
training_acc = 0.71875
epoch = 1
loss = 0.556372880935669
training_acc = 0.75
epoch = 1
loss = 0.42181411385536194
training_acc = 0.8125
epoch = 1
loss = 0.45953354239463806
training_acc = 0.75
epoch = 1
loss = 0.5191757678

 33%|███▎      | 1/3 [01:53<03:47, 113.92s/it]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.53125
epoch = 0
loss = 0.6679942607879639
training_acc = 0.71875
epoch = 0
loss = 0.6315286755561829
training_acc = 0.75
epoch = 0
loss = 0.5502030849456787
training_acc = 0.8125
epoch = 0
loss = 0.46081334352493286
training_acc = 0.625
epoch = 0
loss = 0.617723286151886
training_acc = 0.75
epoch = 0
loss = 0.5633445978164673
training_acc = 0.71875
epoch = 0
loss = 0.5855759382247925
training_acc = 0.71875
epoch = 0
loss = 0.5895566940307617
training_acc = 0.6875
epoch = 0
loss = 0.5119861960411072
training_acc = 0.75
epoch = 0
loss = 0.5461575388908386
training_acc = 0.65625
epoch = 1
loss = 0.5984335541725159
training_acc = 0.9375
epoch = 1
loss = 0.31346479058265686
training_acc = 0.75
epoch = 1
loss = 0.5227802395820618
training_acc = 0.84375
epoch = 1
loss = 0.3355363607406616
training_acc = 0.90625
epoch = 1
loss = 0.29156142473220825
training_acc = 0.875
epoch = 1
loss = 0.365126937

 67%|██████▋   | 2/3 [03:38<01:48, 108.16s/it]

...

 33%|███▎      | 1/3 [01:55<03:50, 115.29s/it]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.46875
epoch = 0
loss = 0.6993608474731445
training_acc = 0.65625
epoch = 0
loss = 0.5996266603469849
training_acc = 0.625
epoch = 0
loss = 0.6327332854270935
training_acc = 0.84375
epoch = 0
loss = 0.5396908521652222
training_acc = 0.71875
epoch = 0
loss = 0.5245877504348755
training_acc = 0.75
epoch = 0
loss = 0.5727294683456421
training_acc = 0.625
epoch = 0
loss = 0.5708596706390381
training_acc = 0.71875
epoch = 0
loss = 0.5753870606422424
training_acc = 0.6875
epoch = 0
loss = 0.4683448374271393
training_acc = 0.84375
epoch = 0
loss = 0.45663654804229736
training_acc = 0.84375
epoch = 1
loss = 0.29896682500839233
training_acc = 0.875
epoch = 1
loss = 0.3873254656791687
training_acc = 0.875
epoch = 1
loss = 0.32093217968940735
training_acc = 0.8125
epoch = 1
loss = 0.4760766625404358
training_acc = 0.625
epoch = 1
loss = 0.6268368363380432
training_acc = 0.75
epoch = 1
loss = 0.5333077

 67%|██████▋   | 2/3 [03:48<01:53, 113.88s/it]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.5
epoch = 0
loss = 0.6906880140304565
training_acc = 0.40625
epoch = 0
loss = 0.6788539290428162
training_acc = 0.5625
epoch = 0
loss = 0.6542096734046936
training_acc = 0.71875
epoch = 0
loss = 0.5230438709259033
training_acc = 0.625
epoch = 0
loss = 0.6727137565612793
training_acc = 0.75
epoch = 0
loss = 0.5458706021308899
training_acc = 0.8125
epoch = 0
loss = 0.4799167513847351
training_acc = 0.65625
epoch = 0
loss = 0.653963029384613
training_acc = 0.65625
epoch = 0
loss = 0.6612672209739685
training_acc = 0.8125
epoch = 0
loss = 0.46671926975250244
training_acc = 0.8125
epoch = 1
loss = 0.4973793029785156
training_acc = 0.8125
epoch = 1
loss = 0.4781404435634613
training_acc = 0.84375
epoch = 1
loss = 0.2466619908809662
training_acc = 0.8125
epoch = 1
loss = 0.5626095533370972
training_acc = 0.9375
epoch = 1
loss = 0.1944170892238617
training_acc = 0.78125
epoch = 1
loss = 0.36954048

100%|██████████| 3/3 [05:33<00:00, 111.19s/it]


In [ ]:
tuning_df

,Num hidden,Learning Rate,Acc,Recall,Precision,F1
0,50,0.0001,0.774533,0.854975,0.654334,0.741319
1,50,0.0005,0.799200,0.875043,0.692142,0.772919
2,50,0.0010,0.810533,0.837974,0.763975,0.799265
3,100,0.0001,0.814667,0.804262,0.825547,0.814765
4,100,0.0005,0.835333,0.841639,0.820956,0.831169
5,100,0.0010,0.825067,0.817530,0.831218,0.824317
6,150,0.0001,0.812000,0.823596,0.788010,0.805410
7,150,0.0005,0.807200,0.866040,0.721037,0.786914
8,150,0.0010,0.809867,0.811491,0.800972,0.806197


### Model specs (aggregation, etc. with best hyperparams)

In [ ]:
aggregation = ["last", "mean"]
bidir = [True, False]

agg_df = []
bidir_df = []
acc_df = []
recall_df = []
precision_df = []
f1_df = []

for agg in aggregation:
  for b in tqdm(bidir):

    model = LSTM_Classifier(n_embed=n_embed, d_embed=d_embed, d_hidden=100, bidirectional = b, aggregation = agg)
    acc, recall, precision, f1 = train(model, train_loader, epochs = 3, lr = 0.0005)
    agg_df.append(agg)
    bidir_df.append(str(b))
    acc_df.append(acc)
    recall_df.append(recall)
    precision_df.append(precision)
    f1_df.append(f1)


tuning_df = pd.DataFrame({"Aggregation method": agg_df, "Bidirectional": bidir_df, "Acc": acc_df, "Recall": recall_df, \
                          "Precision": precision_df, "F1": f1_df})

  0%|          | 0/2 [00:00<?, ?it/s]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.4375
epoch = 0
loss = 0.6064784526824951
training_acc = 0.65625
epoch = 0
loss = 0.599441409111023
training_acc = 0.65625
epoch = 0
loss = 0.559699296951294
training_acc = 0.78125
epoch = 0
loss = 0.4749980568885803
training_acc = 0.8125
epoch = 0
loss = 0.3545178174972534
training_acc = 0.84375
epoch = 0
loss = 0.4085998833179474
training_acc = 0.59375
epoch = 0
loss = 0.5527654886245728
training_acc = 0.875
epoch = 0
loss = 0.3581087589263916
training_acc = 0.65625
epoch = 0
loss = 0.5049831867218018
training_acc = 0.78125
epoch = 0
loss = 0.4543844759464264
training_acc = 0.84375
epoch = 1
loss = 0.43872082233428955
training_acc = 0.84375
epoch = 1
loss = 0.33764028549194336
training_acc = 0.875
epoch = 1
loss = 0.25495344400405884
training_acc = 0.78125
epoch = 1
loss = 0.4022493064403534
training_acc = 0.875
epoch = 1
loss = 0.26075372099876404
training_acc = 0.96875
epoch = 1
loss = 

 50%|█████     | 1/2 [02:02<02:02, 122.06s/it]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.5625
epoch = 0
loss = 0.6696104407310486
training_acc = 0.71875
epoch = 0
loss = 0.5770918726921082
training_acc = 0.6875
epoch = 0
loss = 0.5725721120834351
training_acc = 0.625
epoch = 0
loss = 0.6578260660171509
training_acc = 0.5625
epoch = 0
loss = 0.6885486841201782
training_acc = 0.65625
epoch = 0
loss = 0.6206914186477661
training_acc = 0.71875
epoch = 0
loss = 0.6249432563781738
training_acc = 0.6875
epoch = 0
loss = 0.6477012634277344
training_acc = 0.5
epoch = 0
loss = 0.8036221265792847
training_acc = 0.5625
epoch = 0
loss = 0.7014882564544678
training_acc = 0.46875
epoch = 1
loss = 0.663866400718689
training_acc = 0.5625
epoch = 1
loss = 0.68958580493927
training_acc = 0.78125
epoch = 1
loss = 0.4979111850261688
training_acc = 0.78125
epoch = 1
loss = 0.5429002642631531
training_acc = 0.8125
epoch = 1
loss = 0.5225524306297302
training_acc = 0.90625
epoch = 1
loss = 0.34040161

  0%|          | 0/2 [00:00<?, ?it/s]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.65625
epoch = 0
loss = 0.5626097321510315
training_acc = 0.75
epoch = 0
loss = 0.4260748326778412
training_acc = 0.875
epoch = 0
loss = 0.34529444575309753
training_acc = 0.78125
epoch = 0
loss = 0.42727673053741455
training_acc = 0.90625
epoch = 0
loss = 0.2947957515716553
training_acc = 0.90625
epoch = 0
loss = 0.43915361166000366
training_acc = 0.78125
epoch = 0
loss = 0.40649980306625366
training_acc = 0.90625
epoch = 0
loss = 0.4162519574165344
training_acc = 0.75
epoch = 0
loss = 0.5339694023132324
training_acc = 0.8125
epoch = 0
loss = 0.32581913471221924
training_acc = 0.84375
epoch = 1
loss = 0.3274221122264862
training_acc = 0.8125
epoch = 1
loss = 0.48227906227111816
training_acc = 0.96875
epoch = 1
loss = 0.2090284377336502
training_acc = 0.90625
epoch = 1
loss = 0.23738627135753632
training_acc = 0.90625
epoch = 1
loss = 0.19177664816379547
training_acc = 0.78125
epoch = 1
los

 50%|█████     | 1/2 [02:07<02:07, 127.79s/it]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.59375
epoch = 0
loss = 0.5520038604736328
training_acc = 0.84375
epoch = 0
loss = 0.4354221522808075
training_acc = 0.8125
epoch = 0
loss = 0.3955550193786621
training_acc = 0.8125
epoch = 0
loss = 0.47529271245002747
training_acc = 0.84375
epoch = 0
loss = 0.48612844944000244
training_acc = 0.90625
epoch = 0
loss = 0.16933125257492065
training_acc = 0.875
epoch = 0
loss = 0.3607003390789032
training_acc = 0.8125
epoch = 0
loss = 0.43701615929603577
training_acc = 0.90625
epoch = 0
loss = 0.3181833028793335
training_acc = 0.8125
epoch = 0
loss = 0.3421694040298462
training_acc = 0.90625
epoch = 1
loss = 0.1482355296611786
training_acc = 0.96875
epoch = 1
loss = 0.23473167419433594
training_acc = 0.90625
epoch = 1
loss = 0.19327867031097412
training_acc = 0.90625
epoch = 1
loss = 0.31018635630607605
training_acc = 0.9375
epoch = 1
loss = 0.2562289834022522
training_acc = 0.96875
epoch = 1
l

100%|██████████| 2/2 [04:03<00:00, 121.80s/it]


In [ ]:
tuning_df

,Aggregation method,Bidirectional,Acc,Recall,Precision,F1
0,last,True,0.825733,0.876020,0.753713,0.810277
1,last,False,0.802400,0.866381,0.709155,0.779923
2,mean,True,0.836800,0.854042,0.807453,0.830094
3,mean,False,0.830933,0.846570,0.803133,0.824279


### Trainable vs non trainable; randomize vs non randomize

In [ ]:
trainable = [True, False]
randomize = [True, False]

trainable_df = []
randomize_df = []
acc_df = []
recall_df = []
precision_df = []
f1_df = []

for t in trainable:
  for r in tqdm(randomize):
    embedding_matrix = get_embedding_matrix(n_embed, d_embed, glove_map, randomize_init = r)
    model = LSTM_Classifier(n_embed=n_embed, d_embed=d_embed, freeze_emb = t, d_hidden=100, bidirectional = True, aggregation = "mean")
    acc, recall, precision, f1 = train(model, train_loader, epochs = 3, lr = 0.0005)

    trainable_df.append(str(t))
    randomize_df.append(str(r))
    acc_df.append(acc)
    recall_df.append(recall)
    precision_df.append(precision)
    f1_df.append(f1)


trainable_df = pd.DataFrame({"Trainable embeddings": trainable_df, "Random embed": randomize_df, "Acc": acc_df, "Recall": recall_df, \
                          "Precision": precision_df, "F1": f1_df})

  0%|          | 0/2 [00:00<?, ?it/s]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.46875
epoch = 0
loss = 0.6701143383979797
training_acc = 0.6875
epoch = 0
loss = 0.6133744716644287
training_acc = 0.6875
epoch = 0
loss = 0.5209125280380249
training_acc = 0.71875
epoch = 0
loss = 0.5602103471755981
training_acc = 0.71875
epoch = 0
loss = 0.5057514905929565
training_acc = 0.59375
epoch = 0
loss = 0.540002703666687
training_acc = 0.78125
epoch = 0
loss = 0.4834231734275818
training_acc = 0.75
epoch = 0
loss = 0.4671366214752197
training_acc = 0.8125
epoch = 0
loss = 0.43439245223999023
training_acc = 0.8125
epoch = 0
loss = 0.4028088450431824
training_acc = 0.90625
epoch = 1
loss = 0.27547985315322876
training_acc = 0.8125
epoch = 1
loss = 0.3705717921257019
training_acc = 0.84375
epoch = 1
loss = 0.5243800282478333
training_acc = 0.625
epoch = 1
loss = 0.5198699235916138
training_acc = 0.875
epoch = 1
loss = 0.33950236439704895
training_acc = 0.8125
epoch = 1
loss = 0.259

100%|██████████| 220637/220637 [00:05<00:00, 38929.84it/s]


epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.53125
epoch = 0
loss = 0.6391174793243408
training_acc = 0.875
epoch = 0
loss = 0.4865061044692993
training_acc = 0.71875
epoch = 0
loss = 0.4462340176105499
training_acc = 0.78125
epoch = 0
loss = 0.5007859468460083
training_acc = 0.71875
epoch = 0
loss = 0.48440122604370117
training_acc = 0.78125
epoch = 0
loss = 0.43692779541015625
training_acc = 0.71875
epoch = 0
loss = 0.5206706523895264
training_acc = 0.75
epoch = 0
loss = 0.5791881084442139
training_acc = 0.8125
epoch = 0
loss = 0.28936827182769775
training_acc = 0.90625
epoch = 0
loss = 0.35732242465019226
training_acc = 0.65625
epoch = 1
loss = 0.45880386233329773
training_acc = 0.65625
epoch = 1
loss = 0.5911997556686401
training_acc = 0.875
epoch = 1
loss = 0.2756249010562897
training_acc = 0.8125
epoch = 1
loss = 0.3715904951095581
training_acc = 0.75
epoch = 1
loss = 0.3191026449203491
training_acc = 0.8125
epoch = 1
loss = 0.

  0%|          | 0/2 [00:00<?, ?it/s]

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.625
epoch = 0
loss = 0.7339345812797546
training_acc = 0.75
epoch = 0
loss = 0.5381876230239868
training_acc = 0.65625
epoch = 0
loss = 0.5522716641426086
training_acc = 0.65625
epoch = 0
loss = 0.5919142365455627
training_acc = 0.78125
epoch = 0
loss = 0.3938015103340149
training_acc = 0.75
epoch = 0
loss = 0.5190696120262146
training_acc = 0.875
epoch = 0
loss = 0.36800265312194824
training_acc = 0.625
epoch = 0
loss = 0.544620156288147
training_acc = 0.875
epoch = 0
loss = 0.22900143265724182
training_acc = 0.6875
epoch = 0
loss = 0.464569091796875
training_acc = 0.65625
epoch = 1
loss = 0.4953234791755676
training_acc = 0.6875
epoch = 1
loss = 0.39475366473197937
training_acc = 0.84375
epoch = 1
loss = 0.32259196043014526
training_acc = 0.8125
epoch = 1
loss = 0.3479844331741333
training_acc = 0.875
epoch = 1
loss = 0.3189203143119812
training_acc = 0.84375
epoch = 1
loss = 0.324896395

100%|██████████| 220637/220637 [00:04<00:00, 46263.04it/s]


epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.75
epoch = 0
loss = 0.503365159034729
training_acc = 0.6875
epoch = 0
loss = 0.4766038656234741
training_acc = 0.78125
epoch = 0
loss = 0.3549391031265259
training_acc = 0.75
epoch = 0
loss = 0.6041033267974854
training_acc = 0.71875
epoch = 0
loss = 0.5494006276130676
training_acc = 0.6875
epoch = 0
loss = 0.5245348215103149
training_acc = 0.875
epoch = 0
loss = 0.3071781098842621
training_acc = 0.96875
epoch = 0
loss = 0.2186228632926941
training_acc = 0.71875
epoch = 0
loss = 0.44789236783981323
training_acc = 0.9375
epoch = 0
loss = 0.26760366559028625
training_acc = 0.90625
epoch = 1
loss = 0.3069625496864319
training_acc = 1.0
epoch = 1
loss = 0.1483556628227234
training_acc = 0.875
epoch = 1
loss = 0.366039901971817
training_acc = 0.78125
epoch = 1
loss = 0.42392951250076294
training_acc = 0.90625
epoch = 1
loss = 0.3030067980289459
training_acc = 0.75
epoch = 1
loss = 0.38511347770

100%|██████████| 2/2 [04:20<00:00, 130.23s/it]


In [ ]:
trainable_df

,Trainable embeddings,Random embed,Acc,Recall,Precision,F1
0,True,True,0.811733,0.820599,0.791790,0.805937
1,True,False,0.803467,0.895915,0.681069,0.773857
2,False,True,0.827867,0.823498,0.829058,0.826268
3,False,False,0.827867,0.851399,0.789090,0.819061


In [ ]:
embedding_matrix = get_embedding_matrix(n_embed, d_embed, glove_map, randomize_init = True)
model = LSTM_Classifier(n_embed=n_embed, d_embed=d_embed, freeze_emb = False, d_hidden=100, bidirectional = True, aggregation = "mean")
acc, recall, precision, f1 = train(model, train_loader, epochs = 3, lr = 0.0005, val = "test")


epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.625
epoch = 0
loss = 0.6376409530639648
training_acc = 0.5
epoch = 0
loss = 0.7217854261398315
training_acc = 0.5625
epoch = 0
loss = 0.6285431385040283
training_acc = 0.65625
epoch = 0
loss = 0.6162084341049194
training_acc = 0.8125
epoch = 0
loss = 0.47493666410446167
training_acc = 0.78125
epoch = 0
loss = 0.6035162806510925
training_acc = 0.75
epoch = 0
loss = 0.5329895615577698
training_acc = 0.625
epoch = 0
loss = 0.5700165033340454
training_acc = 0.9375
epoch = 0
loss = 0.20429456233978271
training_acc = 0.84375
epoch = 0
loss = 0.36811134219169617
training_acc = 0.65625
epoch = 1
loss = 0.5197693109512329
training_acc = 0.90625
epoch = 1
loss = 0.2509536147117615
training_acc = 0.75
epoch = 1
loss = 0.5137096047401428
training_acc = 0.90625
epoch = 1
loss = 0.24769623577594757
training_acc = 0.96875
epoch = 1
loss = 0.22999393939971924
training_acc = 0.9375
epoch = 1
loss = 0.27758

RuntimeError: ignored

In [ ]:
with torch.no_grad():
  model.eval()
  y_pred = model(torch.from_numpy(X_test).to('cuda'), seq_l_test)
  y_pred[y_pred >= 0.5] = 1
  y_pred[y_pred < 0.5] = 0
  acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_test).view(-1, 1))

RuntimeError: ignored

In [ ]:
acc, recall, precision, f1

(0.8348, 0.8582496413199426, 0.8005888650963597, 0.8284171167428334)

### Dataset shifts

Training on short words and testing on others

training on short sent and testing on others

In [ ]:
reviews_df['sent_len'] = reviews_df.review.apply(lambda x: len(x.split(' ')))


In [ ]:
np.median(list(reviews_df.sent_len))

169.0

In [ ]:
train_df = reviews_df[reviews_df.sent_len < 200]
test_df = reviews_df[reviews_df.sent_len > 200]

In [ ]:
#Lets transform data to padded versions

max_len_train = max(list(train_df.review.apply(lambda x: len(x.split(' ')))))
max_len_test = max(list(test_df.review.apply(lambda x: len(x.split(' ')))))


X_padded_train = np.zeros((len(train_df), max_len_train + 1))
X_padded_test = np.zeros((len(test_df), max_len_test + 1))

y_train = np.asarray(train_df.sentiment.tolist())
y_test = np.asarray(test_df.sentiment.tolist())

seq_l_train = np.zeros(len(train_df)).astype(int)
seq_l_test = np.zeros(len(test_df)).astype(int)


for i in tqdm(range(len(train_df))):
  padded_arr = convert_to_padded(train_df.iloc[i].review, max_len_train, trim_len = max_len_train)
  X_padded_train[i] = padded_arr

for i in tqdm(range(len(test_df))):
  padded_arr = convert_to_padded(test_df.iloc[i].review, max_len_test, trim_len = max_len_test)
  X_padded_test[i] = padded_arr



100%|██████████| 19692/19692 [00:08<00:00, 2228.59it/s]


In [ ]:
X_padded_train = X_padded_train.astype(int)
X_padded_test = X_padded_test.astype(int)

In [ ]:
seq_l_train = X_padded_train[:, -1]
seq_l_test = X_padded_test[:, -1]

X_train = X_padded_train[:, :-1]
X_test = X_padded_test[:, :-1]

In [ ]:
print(X_train.shape)
print(X_test.shape)

(30187, 199)
(19692, 2441)


In [ ]:
X_test_use = X_test[:1000]

In [ ]:
train_dataset = reviewPytorchDataset(X_train, y_train, seq_l_train)
# test_dataset = reviewPytorchDataset(X_test_use, y_test[:1000], seq_l_test[:1000])

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True, num_workers = 0)
# test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False, num_workers = 0)


In [ ]:
model = LSTM_Classifier(n_embed=n_embed, d_embed=d_embed, d_hidden=100, bidirectional = True, aggregation = "mean")
train(model, train_loader, epochs = 3, lr = .0005, val = "test")

epoch |   %        |  loss  |  avg   |val loss|   acc   |  best  | time | save |
training_acc = 0.625
epoch = 0
loss = 0.6066147089004517
training_acc = 0.625
epoch = 0
loss = 0.45439794659614563
training_acc = 0.9375
epoch = 0
loss = 0.3763020932674408
training_acc = 0.375
epoch = 0
loss = 0.5875682234764099
training_acc = 0.625
epoch = 0
loss = 0.5466835498809814
training_acc = 0.6875
epoch = 0
loss = 0.5909993648529053
training_acc = 0.8125
epoch = 0
loss = 0.45334768295288086
training_acc = 0.875
epoch = 0
loss = 0.36704427003860474
training_acc = 0.9375
epoch = 0
loss = 0.2677190899848938
training_acc = 0.5625
epoch = 0
loss = 0.7882990837097168
training_acc = 0.875
epoch = 0
loss = 0.413955956697464
training_acc = 0.875
epoch = 0
loss = 0.36540770530700684
training_acc = 0.9375
epoch = 0
loss = 0.2260204255580902
training_acc = 0.75
epoch = 0
loss = 0.29841721057891846
training_acc = 0.9375
epoch = 0
loss = 0.153676837682724
training_acc = 0.875
epoch = 0
loss = 0.422017395496368

OutOfMemoryError: ignored

In [ ]:
acc_l, recall_l, precision_l, f1_l = [], [], [], []
for i in range(19):

  with torch.no_grad():
      model.eval()

      y_pred = model(torch.from_numpy(X_test[i*1000:(i+1)*1000]).to('cuda'), seq_l_test[i*1000:(i+1)*1000])
      # print(y_pred)
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0

      acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_test[i*1000:(i+1)*1000]).view(-1, 1))
      acc_l.append(acc)
      recall_l.append(recall)
      precision_l.append(precision)
      f1_l.append(f1)
print(np.mean(acc_l), np.mean(recall_l), np.mean(precision_l), np.mean(f1_l))

0.8452631578947368 0.9515324230016775 0.7277059021976452 0.8235266258199412


In [ ]:
acc_l, recall_l, precision_l, f1_l = [], [], [], []
for i in range(30):

  with torch.no_grad():
      model.eval()

      y_pred = model(torch.from_numpy(X_train[i*1000:(i+1)*1000]).to('cuda'), seq_l_train[i*1000:(i+1)*1000])
      # print(y_pred)
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0

      acc, recall, precision, f1 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_train[i*1000:(i+1)*1000]).view(-1, 1))
      acc_l.append(acc)
      recall_l.append(recall)
      precision_l.append(precision)
      f1_l.append(f1)
print(np.mean(acc_l), np.mean(recall_l), np.mean(precision_l), np.mean(f1_l))

0.9920333333333333 0.995489270967592 0.988483098149963 0.9919636387256048


In [ ]:
#For deep learning models
with torch.no_grad():

  y_pred = model(torch.from_numpy(X_test[10000:len(X_test)]).to('cuda'), seq_l_test[10000:len(X_test)])
  y_pred[y_pred >= 0.5] = 1
  y_pred[y_pred < 0.5] = 0
  acc2, recall2, precision2, f12 = evaluate_model(y_pred.cpu().detach().numpy(), torch.from_numpy(y_test[10000:len(X_test)]).view(-1, 1))

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
#For traditional pipeline models random forest and logistic regression
acc1, recall1, precision1, f11

(0.7928, 0.7998774760057178, 0.7819924136554203, 0.7908338380779325)

In [ ]:
acc2, recall2, precision2, f12

(0.786731324803962, 0.7912387340180256, 0.778992983904251, 0.7850681085577623)

In [ ]:
np.mean(seq_l_test)

373.29448507007925

### Synonyms experiment

### What do we get wrong with this model

In [ ]:
preds = y_pred.cpu().numpy().reshape(1, -1)[0]

corr = []
uncorr = []
fn = []

for i in range(len(preds)):
  if preds[i] == y_test[i]:
    corr.append(i)
  else:
    uncorr.append(i)

    if preds[i] == 0 and y_test[1] == 1:
      fn.append(i)

In [ ]:
X_corr = X_test[corr]
X_uncorr = X_test[uncorr]
X_fn = X_test[fn]

In [ ]:
X_corr_word = []
for a in X_corr:
  X_corr_word.append([id_to_word[x] for x in a])


X_uncorr_word = []
for a in X_uncorr:
  X_uncorr_word.append([id_to_word[x] for x in a])

fn_word = []

for a in X_fn:
  fn_word.append([id_to_word[x] for x in a])

In [ ]:
X_corr_word

NameError: ignored

In [ ]:
print(np.mean([len(x) for x in X_corr_word]))
print(np.mean([len(x) for x in X_uncorr_word]))

100.0
100.0


In [ ]:
def avg_str_length(string):
  l = [len(x) for x in string]
  return np.mean(l)

X_corr_len = [avg_str_length(x) for x in X_corr_word]
X_uncorr_len = [avg_str_length(x) for x in X_uncorr_word]

In [ ]:
np.mean(X_corr_len)

4.42051429484108

In [ ]:
np.mean(X_uncorr_len)

4.452429378531074

In [ ]:
X_uncorr_word

[['compared',
  'to',
  'the',
  'acclaimed',
  'hollywood',
  'remake',
  'this',
  'film',
  'is',
  'less',
  'flashy',
  'but',
  'much',
  'stronger',
  'at',
  'providing',
  'an',
  'overall',
  'picture',
  'of',
  'the',
  'drug',
  'problem',
  'the',
  'remake',
  'loses',
  'the',
  'most',
  'interesting',
  'of',
  'the',
  'three',
  'plot',
  'threads',
  'that',
  'of',
  'a',
  'farmer',
  'whose',
  'meager',
  'livelihood',
  'depends',
  'on',
  'drugs',
  'as',
  'a',
  'paying',
  'cash',
  'crop',
  'and',
  'whose',
  'increased',
  'involvement',
  'with',
  'the',
  'drug',
  'trade',
  'in',
  'an',
  'effort',
  'to',
  'better',
  'himself',
  'destroys',
  'himself',
  'and',
  'his',
  'family',
  'additionally',
  'the',
  'story',
  'of',
  'the',
  'daughters',
  'addiction',
  'goes',
  'into',
  'the',
  'problems',
  'not',
  'only',
  'of',
  'withdrawal',
  'but',
  'the',
  'high',
  'likelihood',
  'of',
  'relapse',
  'this',
  'original',
  '

In [ ]:
reviews_df.groupby("sentiment")["review"].count()

sentiment
0    25000
1    25000
Name: review, dtype: int64

In [ ]:
fn_word[2]

['this',
 'movie',
 'is',
 'about',
 'the',
 'crew',
 'of',
 'a',
 'spaceship',
 'who',
 'crash',
 'land',
 'onto',
 'a',
 'strange',
 'bright',
 'planet',
 'with',
 'three',
 'suns',
 'among',
 'the',
 'passengers',
 'is',
 'convicted',
 'serial',
 'killer',
 'richard',
 'riddick',
 'vin',
 'diesel',
 'and',
 'his',
 'nemesis',
 'johns',
 'cole',
 'hauser',
 'while',
 'the',
 'survivors',
 'of',
 'the',
 'wreck',
 'are',
 'getting',
 'their',
 'bearings',
 'an',
 'eclipse',
 'of',
 'all',
 'three',
 'suns',
 'happens',
 'shortly',
 'afterwards',
 'scary',
 'creatures',
 'begin',
 'to',
 'appear',
 'and',
 'start',
 'to',
 'pick',
 'off',
 'the',
 'crew',
 'members',
 'one',
 'by',
 'one',
 'during',
 'the',
 'eclipse',
 'riddick',
 'comes',
 'into',
 'his',
 'own',
 'as',
 'his',
 'eyeballs',
 'have',
 'been',
 'surgically',
 'shined',
 'giving',
 'him',
 'night',
 'vision',
 'the',
 'crew',
 'have',
 'to',
 'rely',
 'on',
 'him',
 'to',
 'try']

In [ ]:
from nltk.corpus import wordnet
nltk.download('stopwords')
reviews_df_syn = reviews_df.copy()
stop_words = stopwords.words("english")


reviews_df_syn['review'] = reviews_df_syn['review'].apply(lambda x: ' '.join([word if word in stop_words else \
                                                          random.choice(random.choice(list(wordnet.synsets(word))).lemma_names()) \
                                                              if random.random() < 0.3 and len(list(wordnet.synsets(word))) > 0 else word for word in x.split(' ')]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
reviews_df_syn.head()

,Unnamed: 0,review,sentiment
0,0,one of the other reader has mentioned that aft...,1
1,1,a wonderful little production the filming tech...,1
2,2,i thought this was a wonderful path to spend t...,1
3,3,basically there a family where a small boy jak...,0
4,4,petter matteis passion in the time of money is...,1


In [ ]:
reviews_df.head()

,Unnamed: 0,review,sentiment
0,0,one of the other reviewers has mentioned that ...,1
1,1,a wonderful little production the filming tech...,1
2,2,i thought this was a wonderful way to spend ti...,1
3,3,basically theres a family where a little boy j...,0
4,4,petter matteis love in the time of money is a ...,1


In [ ]:
embedding_matrix.shape



(220637, 300)

In [ ]:
a = []
a.append([1, 2, 3])
a.append([3, 44, 5])
a

[[1, 2, 3], [3, 44, 5]]

In [ ]:
word = 'great'
random.choice(random.choice(list(wordnet.synsets(word))).lemma_names())

'great'

In [ ]:
with open('/content/drive/MyDrive/519_temp/glove.840B.300d.txt','rb') as fi:
    for l in tqdm(fi):
        try:
            #### STUDENT CODE HERE ####
            ## only include vectors that are found in the vocabulary.
            line_split = str(l.decode('utf8')).split(' ')

            if line_split[0] not in vocab:
              glove_map[line_split[0]] = [np.float(v) for v in line_split[1:]]
            #### STUDENT CODE ENDS HERE ####
        except:
            #some lines have urls, we don't need them.
            pass

0it [00:00, ?it/s]<ipython-input-90-432ce59802e5>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  glove_map[line_split[0]] = [np.float(v) for v in line_split[1:]]
423031it [03:59, 291.85it/s]

In [ ]:
"what" in glove_map

NameError: ignored